# VectorStore Memory

`VectorStoreMemory` stores interactions in a VectorDB and queries the top-K most "salient" interactions every type it is called.

This differs from most of the other Memory classes in that 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreMemory

### Initialize your VectorStore

Depending on the store you choose, this step may look different. Consult the relevant VectorStore documentation for more details.

In [2]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

### Create your the VectorStoreMemory

In [3]:
memory = VectorStoreMemory(
    vectorstore=vectorstore,
    return_docs=False, # Makes load_memory_variables() return Dict[str, str] instead of Dict[str, List[Document]]
    )

# Examples of how
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "Heard of France?"}, {"output": "Undoubtedly"})
memory.save_context({"input": "How about Australia?"}, {"output": "Most certainly"})
memory.save_context({"input": "What about Montreal?"}, {"output": "Of course."})

In [4]:
# Notice the first result discusses France, which the language model deems more semantically relevant
# than the other locations
print(memory.load_memory_variables({"prompt": "Emmanuel Macron"})["history"])

input: Heard of France?
output: Undoubtedly
input: What about Montreal?
output: Of course.
input: How about Australia?
output: Most certainly
input: hi
output: whats up


## Using in a chain
Let's walk through an example, again setting `verbose=True` so we can see the prompt.

In [5]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
llm = OpenAI() # Can be any valid LLM
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: hi
output: whats up
input: What about Montreal?
output: Of course.
input: How about Australia?
output: Most certainly
input: Heard of France?
output: Undoubtedly
Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. How can I help you?"

In [6]:
conversation_with_summary.predict(input="What do you know about Uluru?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: How about Australia?
output: Most certainly
input: Heard of France?
output: Undoubtedly
input: hi
output: whats up
input: Hi, what's up?
response:  Hi there! I'm doing great. How can I help you?
Human: What do you know about Uluru?
AI:

> Finished chain.


" Uluru is a large sandstone monolith in Australia's Northern Territory. It is also known by its English name Ayers Rock. It is considered to be sacred to the Aboriginal people and is a popular tourist destination."

In [7]:
conversation_with_summary.predict(input="Tell me more about UNESCO!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Heard of France?
output: Undoubtedly
input: What about Montreal?
output: Of course.
input: What do you know about Uluru?
response:  Uluru is a large sandstone monolith in Australia's Northern Territory. It is also known by its English name Ayers Rock. It is considered to be sacred to the Aboriginal people and is a popular tourist destination.
input: How about Australia?
output: Most certainly
Human: Tell me more about UNESCO!
AI:

> Finished chain.


' UNESCO is the United Nations Educational, Scientific, and Cultural Organization. It was founded in 1945 and works to promote international cooperation in the fields of education, science, and culture. UNESCO works to advance global understanding and respect for diversity, cultural heritage, and human rights.'

In [8]:
# We can see here that the buffer is updated
conversation_with_summary.predict(input="Are there UNESCO sites in Canada?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
input: Tell me more about UNESCO!
response:  UNESCO is the United Nations Educational, Scientific, and Cultural Organization. It was founded in 1945 and works to promote international cooperation in the fields of education, science, and culture. UNESCO works to advance global understanding and respect for diversity, cultural heritage, and human rights.
input: What about Montreal?
output: Of course.
input: Heard of France?
output: Undoubtedly
input: How about Australia?
output: Most certainly
Human: Are there UNESCO sites in Canada?
AI:

> Finished chain.


' Yes, there are 18 UNESCO World Heritage Sites in Canada. They include the Canadian Rocky Mountain Parks, the Historic District of Old Québec, the Nahanni National Park Reserve, the Rideau Canal, and the L’Anse aux Meadows National Historic Site.'